# Intro

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.distributions.binomial as binomial

from torch.nn.functional import conv2d, max_pool2d, cross_entropy

from time import time # for time measurement

plt.rc("figure", dpi=100)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(device)
print(device)

## Input data

In [ ]:
batch_size = 100

# transform images into normalized tensors
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,))
])

train_dataset = datasets.MNIST(
    "./",
    download=True,
    train=True,
    transform=transform,
)

test_dataset = datasets.MNIST(
    "./",
    download=True,
    train=False,
    transform=transform,
)


# # Reduce train & test sets for testing the procedure
# train_dataset = torch.utils.data.Subset(train_dataset, list(range(6000)))
# test_dataset = torch.utils.data.Subset(test_dataset, list(range(1000)))


train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=1,
    pin_memory=True,
)

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=1,
    pin_memory=True,
)

## Utility functions

In [ ]:
def init_weights(shape):
    # Kaiming He initialization (a good initialization is important)
    # https://arxiv.org/abs/1502.01852
    std = np.sqrt(2. / shape[0])
    w = torch.randn(size=shape, device=device) * std
    w.requires_grad = True
    
    return w


def rectify(x):
    # Rectified Linear Unit (ReLU)
    return torch.max(torch.zeros_like(x), x)


def dropout(X, p_drop=0.5):
    if not (0 < p_drop < 1): return X
    phi = binomial.Binomial(torch.ones(X.shape), 1 - p_drop).sample()
    
    return phi.to(device)*X / (1-p_drop)


def PRelu(X, a):
    '''
    Parametric ReLU, i.e. returns a copy of 'X' where all negative entries have been
    multiplied with the corresponding entries in 'a'.
    '''
    return ((X>=0) + (X<0)*a) * X


def convolute(X, w, p_drop):
    '''
    Performs convolution, ReLU, subsampling and dropout
    '''
    conv = conv2d(X, w)
    rect = rectify(conv)
    subs = max_pool2d(rect, (2, 2))
    drop = dropout(subs, p_drop)

    return drop


class RMSprop(optim.Optimizer):
    """
    This is a reduced version of the PyTorch internal RMSprop optimizer
    It serves here as an example
    """
    def __init__(self, params, lr=1e-3, alpha=0.5, eps=1e-8):
        defaults = dict(lr=lr, alpha=alpha, eps=eps)
        super(RMSprop, self).__init__(params, defaults)

    def step(self):
        for group in self.param_groups:
            for p in group['params']:
                grad = p.grad.data
                state = self.state[p]

                # state initialization
                if len(state) == 0:
                    state['square_avg'] = torch.zeros_like(p.data)

                square_avg = state['square_avg']
                alpha = group['alpha']

                # update running averages
                square_avg.mul_(alpha).addcmul_(grad, grad, value=1 - alpha)
                avg = square_avg.sqrt().add_(group['eps'])

                # gradient update
                p.data.addcdiv_(grad, avg, value=-group['lr'])

## Models

In [ ]:
# define the CNN
def cnn_model(x, w_c1, w_c2, w_c3, w_h2, w_a2, w_o, p_drop_input, p_drop_hidden):
    c1 = convolute(dropout(x, p_drop_input), w_c1, p_drop_hidden)
    c2 = convolute(c1, w_c2, p_drop_hidden)
    c3 = convolute(c2, w_c3, p_drop_hidden)

    c3 = c3.reshape(-1, 128)

    h2 = dropout(c3 @ w_h2, p_drop_hidden)
    a2 = PRelu(h2, w_a2)
    pre_softmax = a2 @ w_o

    return pre_softmax

## Time keeping

In [ ]:
def time_to_str(time):
    '''
    Converts time (in seconds) to more readable format
    '''
    time = round(time)
    hours = time//3600
    minutes = (time%3600)//60
    seconds = time%60

    return f"{hours:3d}h {minutes:02d}m {seconds:02d}s"


def print_time_info(time_start, time_epoch_start, time, n_epochs, n_drops, i_epoch, i_drop, p_drop):
    dtime_total = time - time_start
    dtime_epoch = time - time_epoch_start
    epochs_done_total = i_drop*n_epochs + i_epoch + 1
    epochs_done_drop = i_epoch + 1
    epochs_total = n_drops*n_epochs
    print(f"Time elapsed: {time_to_str(dtime_total)} | Time remaining (drop={p_drop}): {time_to_str((n_epochs - epochs_done_drop) * dtime_epoch / epochs_done_drop)} | Time remaining (total): {time_to_str((epochs_total - epochs_done_total) * dtime_total / epochs_done_total)}")

## Running the network

Note that the training losses are skewed by dropout (since we're multiplying non-dropped inputs).

In [ ]:
data = {}

In [ ]:
# stuff for keeping and estimating time
time_start = time()
n_drops = 3


for i, p in enumerate([0.1, 0.25, 0.5]):
    print("--------")
    print(f"Running CNN model with dropout {p}")

    time_epoch_start = time()

    w_c1 = init_weights((32, 1, 5, 5))
    w_c2 = init_weights((64, 32, 5, 5))
    w_c3 = init_weights((128, 64, 2, 2))
    w_h2 = init_weights((128, 625))
    w_a2 = init_weights((625,))
    w_o = init_weights((625, 10))

    optimizer = RMSprop(params=[w_c1, w_c2, w_c3, w_h2, w_a2, w_o])

    n_epochs = 100

    train_loss = []
    test_loss = []

    data[p] = (optimizer, train_loss, test_loss) # for later use

    # put this into a training loop over 100 epochs
    for epoch in range(n_epochs + 1):
        train_loss_this_epoch = []
        for idx, batch in enumerate(train_dataloader):
            x, y = batch

            x = x.to(device)
            y = y.to(device)

            # our model requires input of dim (c x w x h)
            x = x.reshape(batch_size, 1, 28, 28)
            # feed input through model
            noise_py_x = cnn_model(x, w_c1, w_c2, w_c3, w_h2, w_a2, w_o, p_drop_input=p, p_drop_hidden=p)

            # reset the gradient
            optimizer.zero_grad()

            # the cross-entropy loss function already contains the softmax
            loss = cross_entropy(noise_py_x, y, reduction="mean")

            train_loss_this_epoch.append(float(loss.to("cpu")))

            # compute the gradient
            loss.backward()

            # update weights
            optimizer.step()

        train_loss.append(np.mean(train_loss_this_epoch))

        # test periodically
        if epoch % 10 == 0:
            print(f"Epoch: {epoch}")
            print(f"Mean Train Loss: {train_loss[-1]:.2e}")
            test_loss_this_epoch = []

            # no need to compute gradients for validation
            with torch.no_grad():
                for idx, batch in enumerate(test_dataloader):
                    x, y = batch

                    x = x.to(device)
                    y = y.to(device)
                    x = x.reshape(batch_size, 1, 28, 28)

                    # no dropouts for testing!
                    noise_py_x = cnn_model(x, w_c1, w_c2, w_c3, w_h2, w_a2, w_o, p_drop_input=0, p_drop_hidden=0)

                    loss = cross_entropy(noise_py_x, y, reduction="mean")
                    test_loss_this_epoch.append(float(loss.to("cpu")))

            test_loss.append(np.mean(test_loss_this_epoch))

            print(f"Mean Test Loss:  {test_loss[-1]:.2e}")
            print_time_info(time_start, time_epoch_start, time(), n_epochs, n_drops, epoch, i, p)

print("\nDone!")


In [ ]:
fig = plt.gcf()
fig.set_size_inches(15, 5)

for i, (p, date) in enumerate(data.items()):
    _, train_loss, test_loss = date
    plt.subplot(1, 3, i+1)
    plt.plot(np.arange(1, n_epochs + 2), train_loss, label=f"Train")
    plt.plot(np.arange(1, n_epochs + 2, 10), test_loss, label="Test")
    plt.title(f"CNN with drop {p}")
    plt.ylim(0, 3)

plt.show()